### 0.1. Imports

In [6]:
import json
import time
import requests
import numpy as np
import pandas as pd

from datetime import datetime
from tweepy import OAuthHandler, Stream, API, Client #StreamListener
from utils import CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET

# 1.0. Twitter Data Collect

## 1.1. Twitter API Request

In [ ]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = API(auth)

In [ ]:
query = 'Brasil'
search_times = 100
client = Client(consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
                       access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET,
                       bearer_token=BEARER_TOKEN, )

tweets = client.search_recent_tweets(query=query, max_results=search_times)

In [ ]:
tick = time.time()
DATE = datetime.now().strftime('%Y-%m-%d')
for k in tweets.data:
    tweet_data = api.get_status(k.id, tweet_mode='extended')._json
    time.sleep(1)
    print(json.dumps(tweet_data) + '\n', file=open(f'output_json_tweets_{DATE}.txt', 'a'))

tock = time.time()
print(f"\nFor {search_times} requests, total time is: {int(tock-tick)} Seconds\n_______________")


For 100 requests, total time is: 127 Seconds
_______________


## 1.2. Data Pre Processing

### 1.2.1. One Example Pre Processing

In [ ]:
with open('output_json_tweets.txt', 'r') as file_jsons:
    data = file_jsons.readlines()

data2 = [json.loads(data[k]) for k in range(0, len(data), 2)]

In [ ]:
# Primary Data in Json
post_author_info = data2[0]['retweeted_status']

post_id = post_author_info['id_str']
post_geo = post_author_info['geo']
post_text = post_author_info['full_text']
post_place = post_author_info['place']
post_source = post_author_info['source']
post_coordinates = post_author_info['coordinates']
post_contributors = post_author_info['contributors']
post_post_author_info_count = post_author_info['retweet_count']
post_favorite_count = post_author_info['favorite_count']
post_is_quote_status = post_author_info['is_quote_status']

post_user_id = post_author_info['user']['id_str']
post_user_name = post_author_info['user']['name']
post_user_lang = post_author_info['user']['lang']
post_user_verified = post_author_info['user']['verified']
post_user_location = post_author_info['user']['location']
post_user_time_zone = post_author_info['user']['time_zone']
post_user_created_at = post_author_info['user']['created_at']
post_user_description = post_author_info['user']['description']
post_user_screen_name = post_author_info['user']['screen_name']

post_user_geo_enabled = post_author_info['user']['geo_enabled']
post_user_friends_count = post_author_info['user']['friends_count']
tweet_user_profile_color = post_author_info['user']['profile_link_color']
post_user_followers_count = post_author_info['user']['followers_count']
post_user_favourites_count = post_author_info['user']['favourites_count']
tweet_user_profile_background_color = post_author_info['user']['profile_background_color']


post_hashtags = data2[0]['entities']['hashtags']

In [ ]:
############################################ Retweet Status

user = data2[0]['user'].copy()

tweet_user_id = user['id_str']
tweet_user_name = user['name']
tweet_user_lang = user['lang']
tweet_user_desc = user['description']
tweet_user_c_fllw = user['followers_count']
tweet_user_c_frds = user['friends_count']
tweet_user_c_fav  = user['favourites_count']
tweet_user_timezone = user['time_zone']
tweet_user_location = user['location']
tweet_user_geo_enab = user['geo_enabled']
tweet_user_date_acc = user['created_at']
tweet_user_is_verify = user['verified']
tweet_user_screen_name = user['screen_name']
tweet_user_profile_color = user['profile_link_color']
tweet_user_profile_background_color = user['profile_background_color']


r_post_id = data2[0]['id_str']
r_post_lang = data2[0]['lang']
r_post_is_fav = data2[0]['favorited']
r_post_source = data2[0]['source']
r_post_fav_count = data2[0]['favorite_count']
r_post_is_retweet = data2[0]['retweeted']
r_post_created_at = data2[0]['created_at']
r_post_retweet_count = data2[0]['retweet_count']



In [ ]:
post = [post_id,post_user_id,post_geo,post_text,post_place,post_source,post_coordinates,post_contributors,post_post_author_info_count,post_favorite_count,post_is_quote_status,]
post_cols = ['id', 'user_id', 'geo', 'text', 'place', 'source', 'coordinates', 'contributors', 'retweet_count', 'fav_count', 'is_quote']
df_post = pd.DataFrame(post).T
df_post.columns = post_cols

In [ ]:
user = [post_user_id,post_user_name,post_user_lang,post_user_verified,post_user_location,post_user_time_zone,post_user_created_at,post_user_description,post_user_screen_name,post_user_geo_enabled,post_user_friends_count,tweet_user_profile_color,post_user_followers_count,post_user_favourites_count,tweet_user_profile_background_color]
user_cols = ['id', 'name', 'lang', 'is_verified', 'location', 'time_zone', 'created_at', 'description', 'screen_name', 'geo_enabled', 'friends_count', 'profile_color', 'followers_count', 'favourites_count', 'background_color']
df_user = pd.DataFrame(user).T
df_user.columns = user_cols

r_user = [tweet_user_id,tweet_user_name,tweet_user_lang,tweet_user_is_verify,tweet_user_location,tweet_user_timezone,tweet_user_date_acc,tweet_user_desc,tweet_user_screen_name,tweet_user_geo_enab,tweet_user_c_frds,tweet_user_profile_color,tweet_user_c_fllw,tweet_user_c_fav,tweet_user_profile_background_color]
df_r_user = pd.DataFrame(r_user).T
df_r_user.columns = user_cols
df_user = pd.concat([df_user, df_r_user], axis=0)

In [ ]:
retweets_info = [r_post_id,tweet_user_id,r_post_lang,r_post_is_fav,r_post_source,r_post_fav_count,r_post_is_retweet,r_post_created_at,r_post_retweet_count]
retweets_info_cols = ['post_id', 'retweet_user_id', 'lang', 'is_fav', 'source', 'fav_counts', 'is_retweet', 'created_at', 'retweet_count']
df_retweets = pd.DataFrame(retweets_info).T
df_retweets.columns = retweets_info_cols

### 1.2.2. Multiple Examples

In [ ]:
DATE = '2022-06-26'

In [ ]:
with open(f'output_json_tweets_{DATE}.txt', 'r') as file_jsons:
    data = file_jsons.readlines()

data2 = [json.loads(data[k]) for k in range(0, len(data), 2)]

In [ ]:
COLLECT_DATETIME = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

post_cols = ['id', 'user_id', 'geo', 'text', 'place', 'source', 'coordinates', 'contributors', 'retweet_count', 'fav_count', 'is_quote', 'urls']
user_cols = ['id', 'name', 'lang', 'is_verified', 'location', 'time_zone', 'created_at', 'description', 'screen_name', 'geo_enabled', 'friends_count', 'profile_color', 'followers_count', 'favourites_count', 'background_color']
retweets_info_cols = ['post_id', 'retweet_user_id', 'lang', 'is_fav', 'source', 'fav_counts', 'is_retweet', 'created_at', 'retweet_count']

dataset_posts = pd.DataFrame(columns=post_cols)
dataset_users = pd.DataFrame(columns=user_cols)
dataset_retweets = pd.DataFrame(columns=retweets_info_cols)

for k in range(len(data2)):
    
    try:
        post_author_info = data2[k]['retweeted_status']
        has_url = data2[k]['retweeted_status']['entities']['urls']
    
    except:
        post_autor_info = data2[k]['user']
        data2[k]['entities']['urls']
    
    post_urls = [url['url'] for url in has_url]
    
    if post_urls:
        post_urls = ', '.join([url['url'] for url in has_url])
        
    else:
        post_urls = 'None'
        
    try:
        post_place = post_author_info['place']['full_name']
    except:
        post_palce = 'None'
    
    post_id = post_author_info['id_str']
    post_geo = post_author_info['geo']
    post_text = post_author_info['full_text']
    post_source = post_author_info['source']
    post_coordinates = post_author_info['coordinates']
    post_contributors = post_author_info['contributors']
    post_post_author_info_count = post_author_info['retweet_count']
    post_favorite_count = post_author_info['favorite_count']
    post_is_quote_status = post_author_info['is_quote_status']

    post_user_id = post_author_info['user']['id_str']
    post_user_name = post_author_info['user']['name']
    post_user_lang = post_author_info['user']['lang']
    post_user_verified = post_author_info['user']['verified']
    post_user_location = post_author_info['user']['location']
    post_user_time_zone = post_author_info['user']['time_zone']
    post_user_created_at = post_author_info['user']['created_at']
    post_user_description = post_author_info['user']['description']
    post_user_screen_name = post_author_info['user']['screen_name']

    post_user_geo_enabled = post_author_info['user']['geo_enabled']
    post_user_friends_count = post_author_info['user']['friends_count']
    tweet_user_profile_color = post_author_info['user']['profile_link_color']
    post_user_followers_count = post_author_info['user']['followers_count']
    post_user_favourites_count = post_author_info['user']['favourites_count']
    tweet_user_profile_background_color = post_author_info['user']['profile_background_color']


    post_hashtags = data2[k]['entities']['hashtags']


    user = data2[k]['user'].copy()

    tweet_user_id = user['id_str']
    tweet_user_name = user['name']
    tweet_user_lang = user['lang']
    tweet_user_desc = user['description']
    tweet_user_c_fllw = user['followers_count']
    tweet_user_c_frds = user['friends_count']
    tweet_user_c_fav  = user['favourites_count']
    tweet_user_timezone = user['time_zone']
    tweet_user_location = user['location']
    tweet_user_geo_enab = user['geo_enabled']
    tweet_user_date_acc = user['created_at']
    tweet_user_is_verify = user['verified']
    tweet_user_screen_name = user['screen_name']
    tweet_user_profile_color = user['profile_link_color']
    tweet_user_profile_background_color = user['profile_background_color']


    r_post_id = data2[k]['id_str']
    r_post_lang = data2[k]['lang']
    r_post_is_fav = data2[k]['favorited']
    r_post_source = data2[k]['source']
    r_post_fav_count = data2[k]['favorite_count']
    r_post_is_retweet = data2[k]['retweeted']
    r_post_created_at = data2[k]['created_at']
    r_post_retweet_count = data2[k]['retweet_count']


    post = [post_id,post_user_id,post_geo,post_text,post_place,post_source,post_coordinates,post_contributors,post_post_author_info_count,post_favorite_count,post_is_quote_status,post_urls]
    user = [post_user_id,post_user_name,post_user_lang,post_user_verified,post_user_location,post_user_time_zone,post_user_created_at,post_user_description,post_user_screen_name,post_user_geo_enabled,post_user_friends_count,tweet_user_profile_color,post_user_followers_count,post_user_favourites_count,tweet_user_profile_background_color]
    r_user = [tweet_user_id,tweet_user_name,tweet_user_lang,tweet_user_is_verify,tweet_user_location,tweet_user_timezone,tweet_user_date_acc,tweet_user_desc,tweet_user_screen_name,tweet_user_geo_enab,tweet_user_c_frds,tweet_user_profile_color,tweet_user_c_fllw,tweet_user_c_fav,tweet_user_profile_background_color]
    retweets_info = [r_post_id,tweet_user_id,r_post_lang,r_post_is_fav,r_post_source,r_post_fav_count,r_post_is_retweet,r_post_created_at,r_post_retweet_count]
    
    df_post = pd.DataFrame(post).T
    df_post.columns = post_cols
    dataset_posts = pd.concat([dataset_posts, df_post], axis=0)

    df_user = pd.DataFrame(user).T
    df_user.columns = user_cols

    df_r_user = pd.DataFrame(r_user).T
    df_r_user.columns = user_cols
    df_user = pd.concat([df_user, df_r_user], axis=0)
    dataset_users = pd.concat([dataset_users, df_user], axis=0)

    df_retweets = pd.DataFrame(retweets_info).T
    df_retweets.columns = retweets_info_cols
    dataset_retweets = pd.concat([dataset_retweets, df_retweets], axis=0)
    
for date, df in zip([COLLECT_DATETIME]*3, [dataset_posts, dataset_users, dataset_retweets]):
    df['request_datetime'] = date
    df = df.reset_index(drop=True)

In [5]:
#dataset_posts[~dataset_posts['urls'].str.contains('None')].head(2)

In [4]:
#dataset_retweets.head(2)

In [3]:
#dataset_users.head(2) #Hide Data

### 1.2.3. Script Class

In [ ]:
import json
import time
import requests
import numpy as np
import pandas as pd

from datetime import datetime
from tweepy import OAuthHandler, Stream, API, Client
from utils import CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET


SEARCH_TIMES = 100
SEARCH_TERMS = 'Brasil'
DATE = datetime.now().strftime('%Y-%m-%d')
COLLECT_DATETIME = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

client = Client(consumer_key=CONSUMER_KEY, consumer_secret=CONSUMER_SECRET,
                access_token=ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET,
                bearer_token=BEARER_TOKEN)


class TwitterRequests():
    def __init__(self):
        self.post_cols = ['id', 'user_id', 'geo', 'text', 'place', 'source', 'coordinates', 'contributors', 'retweet_count', 'fav_count', 'is_quote', 'urls']
        self.user_cols = ['id', 'name', 'lang', 'is_verified', 'location', 'time_zone', 'created_at', 'description', 'screen_name', 'geo_enabled', 'friends_count', 'profile_color', 'followers_count', 'favourites_count', 'background_color']
        self.retweets_info_cols = ['post_id', 'retweet_user_id', 'lang', 'is_fav', 'source', 'fav_counts', 'is_retweet', 'created_at', 'retweet_count']

    
    def api_request(self):
        tweets = client.search_recent_tweets(query=SEARCH_TERMS, max_results=SEARCH_TIMES)

        for k in tweets.data:
            tweet_data = api.get_status(k.id, tweet_mode='extended')._json
            time.sleep(1)
            print(json.dumps(tweet_data) + '\n', file=open(f'output_json_tweets_{DATE}.txt', 'a'))
            
    def load_data(self):
        with open(f'output_json_tweets_{DATE}.txt', 'r') as file_jsons:
            data = file_jsons.readlines()

        data_jsons = [json.loads(data[k]) for k in range(0, len(data), 2)]

        return data_jsons
    
    def get_datasets(self, data_jsons):
        dataset_posts = pd.DataFrame(columns=self.post_cols)
        dataset_users = pd.DataFrame(columns=self.user_cols)
        dataset_retweets = pd.DataFrame(columns=self.retweets_info_cols)

        for k in range(len(data_jsons)):
            try:
                post_author_info = data_jsons[k]['retweeted_status']
                has_url = data_jsons[k]['retweeted_status']['entities']['urls']

            except:
                post_autor_info = data_jsons[k]['user']
                data_jsons[k]['entities']['urls']

            post_urls = [url['url'] for url in has_url]

            if post_urls: post_urls = ', '.join([url['url'] for url in has_url])
            else: post_urls = 'None'

            post_id = post_author_info['id_str']
            post_geo = post_author_info['geo']
            post_text = post_author_info['full_text']
            post_place = post_author_info['place']
            post_source = post_author_info['source']
            post_coordinates = post_author_info['coordinates']
            post_contributors = post_author_info['contributors']
            post_post_author_info_count = post_author_info['retweet_count']
            post_favorite_count = post_author_info['favorite_count']
            post_is_quote_status = post_author_info['is_quote_status']

            post_user_id = post_author_info['user']['id_str']
            post_user_name = post_author_info['user']['name']
            post_user_lang = post_author_info['user']['lang']
            post_user_verified = post_author_info['user']['verified']
            post_user_location = post_author_info['user']['location']
            post_user_time_zone = post_author_info['user']['time_zone']
            post_user_created_at = post_author_info['user']['created_at']
            post_user_description = post_author_info['user']['description']
            post_user_screen_name = post_author_info['user']['screen_name']

            post_user_geo_enabled = post_author_info['user']['geo_enabled']
            post_user_friends_count = post_author_info['user']['friends_count']
            tweet_user_profile_color = post_author_info['user']['profile_link_color']
            post_user_followers_count = post_author_info['user']['followers_count']
            post_user_favourites_count = post_author_info['user']['favourites_count']
            tweet_user_profile_background_color = post_author_info['user']['profile_background_color']


            post_hashtags = data_jsons[k]['entities']['hashtags']

            user = data_jsons[k]['user'].copy()

            tweet_user_id = user['id_str']
            tweet_user_name = user['name']
            tweet_user_lang = user['lang']
            tweet_user_desc = user['description']
            tweet_user_c_fllw = user['followers_count']
            tweet_user_c_frds = user['friends_count']
            tweet_user_c_fav  = user['favourites_count']
            tweet_user_timezone = user['time_zone']
            tweet_user_location = user['location']
            tweet_user_geo_enab = user['geo_enabled']
            tweet_user_date_acc = user['created_at']
            tweet_user_is_verify = user['verified']
            tweet_user_screen_name = user['screen_name']
            tweet_user_profile_color = user['profile_link_color']
            tweet_user_profile_background_color = user['profile_background_color']


            r_post_id = data_jsons[k]['id_str']
            r_post_lang = data_jsons[k]['lang']
            r_post_is_fav = data_jsons[k]['favorited']
            r_post_source = data_jsons[k]['source']
            r_post_fav_count = data_jsons[k]['favorite_count']
            r_post_is_retweet = data_jsons[k]['retweeted']
            r_post_created_at = data_jsons[k]['created_at']
            r_post_retweet_count = data_jsons[k]['retweet_count']


            post = [post_id,post_user_id,post_geo,post_text,post_place,post_source,post_coordinates,post_contributors,post_post_author_info_count,post_favorite_count,post_is_quote_status,post_urls]
            user = [post_user_id,post_user_name,post_user_lang,post_user_verified,post_user_location,post_user_time_zone,post_user_created_at,post_user_description,post_user_screen_name,post_user_geo_enabled,post_user_friends_count,tweet_user_profile_color,post_user_followers_count,post_user_favourites_count,tweet_user_profile_background_color]
            r_user = [tweet_user_id,tweet_user_name,tweet_user_lang,tweet_user_is_verify,tweet_user_location,tweet_user_timezone,tweet_user_date_acc,tweet_user_desc,tweet_user_screen_name,tweet_user_geo_enab,tweet_user_c_frds,tweet_user_profile_color,tweet_user_c_fllw,tweet_user_c_fav,tweet_user_profile_background_color]
            retweets_info = [r_post_id,tweet_user_id,r_post_lang,r_post_is_fav,r_post_source,r_post_fav_count,r_post_is_retweet,r_post_created_at,r_post_retweet_count]

            df_post = pd.DataFrame(post).T
            df_post.columns = self.post_cols
            dataset_posts = pd.concat([dataset_posts, df_post], axis=0)

            df_user = pd.DataFrame(user).T
            df_user.columns = self.user_cols

            df_r_user = pd.DataFrame(r_user).T
            df_r_user.columns = self.user_cols
            df_user = pd.concat([df_user, df_r_user], axis=0)
            dataset_users = pd.concat([dataset_users, df_user], axis=0)

            df_retweets = pd.DataFrame(retweets_info).T
            df_retweets.columns = self.retweets_info_cols
            dataset_retweets = pd.concat([dataset_retweets, df_retweets], axis=0)

            for date, df in zip([COLLECT_DATETIME]*3, [dataset_posts, dataset_users, dataset_retweets]):
                df['request_datetime'] = date
                df = df.reset_index(drop=True)
                
        dataset_posts.to_csv('dataset_posts.csv')
        dataset_users.to_csv('dataset_users.csv')
        dataset_retweets.to_csv('dataset_retweets.csv')
                
        datasets = {'posts': dataset_posts, 
                    'users': dataset_users,
                    'retweets': dataset_retweets} 
        
        return datasets
    

if __name__ == '__main__':
    HOUR_START = datetime.now().strftime('%H:%M:%S')
    print(f'[ API Twitter Request Daily Data ] [{DATE}] -> Time Start at {HOUR_START}', file=open('time_logs.txt', 'a'))
    
    tr = TwitterRequests()
    
    tr.api_request()
    
    dataset_jsons = tr.load_data()
    
    datasets = tr.get_datasets(dataset_jsons)
    
    HOUR_END = datetime.now().strftime('%H:%M:%S')
    print(f'[ API Twitter Request Daily Data ] [{DATE}] -> Time End   at {HOUR_END}\n', file=open('time_logs.txt', 'a'))

## 1.3. Test CSV File

In [2]:
import pandas as pd

#pd.read_csv('data2/dfusers.csv') #Hide Data